### Comparação usando Rapidfuzz entre endereços tratados (base ouro) e brutos (base CNEFE)



In [ ]:
import pandas as pd
from rapidfuzz import process, fuzz
import unidecode

# Normalização de texto
def normalize(text):
    if pd.isna(text):
        return ""
    texto = unidecode.unidecode(str(text)).strip().lower()
    return texto

# Usa um dicionário para substituir abreviações
def normalizar_abreviacoes(texto):
    abreviacoes = {
        " av ": " avenida ",  
        " avn ": " avenida ",
        " r ": " rua ",
        " pc ": " praca ",
        " al ": " alameda ",
        " tr ": " travessa ",
        " jd ": " jardim ",
        " vl ": " vila "
    }
    texto = " " + texto + " " 
    for abrev, completo in abreviacoes.items():
        texto = texto.replace(abrev, completo)
    return texto.strip()  

df_bruto = pd.read_csv("3513801_DIADEMA.csv", sep=";", dtype=str)
df_tratado = pd.read_csv("cnes_geo_padrao_ouro_diadema.csv", sep=";", dtype=str)

### Unificação dos campos dos endereços para comparação

In [ ]:
# Concatenação dos campos da base bruta
df_bruto["endereco_original"] = (
    df_bruto["NOM_TIPO_SEGLOGR"].fillna("") + " " +
    df_bruto["NOM_TITULO_SEGLOGR"].fillna("") + " " +
    df_bruto["NOM_SEGLOGR"].fillna("") + " " +
    df_bruto["NUM_ENDERECO"].fillna("") + " " +
    df_bruto["DSC_LOCALIDADE"].fillna("")
).apply(normalize)

# Concatenação dos campos da base tratada
df_tratado["endereco_original"] = (
    df_tratado["NO_LOGRADO"].fillna("") + " " +
    df_tratado["NU_ENDEREC"].fillna("") + " " +
    df_tratado["NO_BAIRRO"].fillna("")
).apply(normalize)

# Usa a normalização de abreviações
df_bruto["endereco_normalizado"] = df_bruto["endereco_original"].apply(normalizar_abreviacoes)  
df_tratado["endereco_normalizado"] = df_tratado["endereco_original"].apply(normalizar_abreviacoes)  

df_tratado[["endereco_original", "endereco_normalizado"]].head(10)

,endereco_original,endereco_normalizado
0,av antonio piranga 614 centro,avenida antonio piranga 614 centro
1,avenida antonio piranga 700 centro,avenida antonio piranga 700 centro
2,av antonio piranga 614 centro,avenida antonio piranga 614 centro
3,rua dos cariris 195 piraporinha,rua dos cariris 195 piraporinha
4,rua moaciar goulart cunha caldas 111 centro,rua moaciar goulart cunha caldas 111 centro
5,praca rui barbosa 27 piraporinha,praca rui barbosa 27 piraporinha
6,av nossa senhora dos navegantes 288 eldorado,avenida nossa senhora dos navegantes 288 eldorado
7,rua capibaribe 193 jardim campanario,rua capibaribe 193 jardim campanario
8,av piraporinha 1682 piraporinha,avenida piraporinha 1682 piraporinha
9,rua jose bonifacio 1641 serraria,rua jose bonifacio 1641 serraria


### Busca do endereço mais semelhtante usando o RapidFuzz
O 'fuzz.token_set_ratio' ignora a ordem das palavras, trata os casos quando um endereço está contido no outro e quando há palavras extras em um dos lados.

Ex: "Rua das Flores 123 América" vs "Rua das Flores 123 Jardim América"

In [ ]:
resultados = []
for idx_tratado, row_tratado in df_tratado.iterrows():
    endereco_t_normalizado = row_tratado["endereco_normalizado"]  
    endereco_t_original = row_tratado["endereco_original"]        

    match, score, idx_bruto = process.extractOne(
        endereco_t_normalizado,                                  
        df_bruto["endereco_normalizado"],                        
        scorer=fuzz.token_set_ratio         # Há palavras extras em um dos lados, mas a maioria está presente nos dois
    )

    endereco_b_original = df_bruto.loc[idx_bruto, "endereco_original"]

    resultados.append({
        "idx_tratado": idx_tratado,
        "endereco_tratado": endereco_t_original,                
        "idx_bruto": idx_bruto,  
        "endereco_bruto": endereco_b_original,                 
        "similaridade": score,
        "endereco_tratado_normalizado": endereco_t_normalizado,
        "endereco_bruto_normalizado": df_bruto.loc[idx_bruto, "endereco_normalizado"],  
    })

### Visualização dos resultados

In [89]:
df_resultados = pd.DataFrame(resultados)

df_resultados['similaridade'] = df_resultados['similaridade'].astype(str).str.replace('.', ',')

df_resultados[["idx_tratado", 
               "endereco_tratado",   
               "idx_bruto",  
               "endereco_bruto",                 
               "similaridade"]].head(10)

,idx_tratado,endereco_tratado,idx_bruto,endereco_bruto,similaridade
0,0,av antonio piranga 614 centro,39597,avenida antonio piranga 61 centro,"98,50746268656717"
1,1,avenida antonio piranga 700 centro,50282,avenida antonio piranga 700 centro,"100,0"
2,2,av antonio piranga 614 centro,39597,avenida antonio piranga 61 centro,"98,50746268656717"
3,3,rua dos cariris 195 piraporinha,3541,rua dos cariris 195 piraporinha,"100,0"
4,4,rua moaciar goulart cunha caldas 111 centro,70850,rua moacyr goulart cunha caldas 111 conceicao,"86,36363636363636"
5,5,praca rui barbosa 27 piraporinha,160130,praca rui barbosa 27 piraporinha,"100,0"
6,6,av nossa senhora dos navegantes 288 eldorado,43091,avenida nossa senhora dos navegantes 289 eldorado,"97,95918367346938"
7,7,rua capibaribe 193 jardim campanario,115200,rua capibaribe 193 campanario,"100,0"
8,8,av piraporinha 1682 piraporinha,30100,avenida piraporinha 1682 piraporinha,"100,0"
9,9,rua jose bonifacio 1641 serraria,74232,rua jose bonifacio 1641 serraria,"100,0"


### Exporta os resultados em um arquivo CSV

In [88]:
df_resultados.to_csv("enderecos_comparados_tratado_vs_bruto.csv", sep=";", index=False)